# Stacks and Queues

In this week we will see two fundamental data types: **stacks** and **queues**. In each data type we must be able to perform some operations such as *insert*, *remove*, *iterate* and *test* if it is empty. Below we can see the differences between the two data types:

<img src="https://cdn.rawgit.com/rogergranada/MOOCs/master/Coursera/Princeton/Algorithms-Part-1/Week%202/images/datatypes.svg" width="90%" align="center"/>

## Stack

As the image illustrates, in the Stack we have two operation to insert and remove an item, called *push* and *pop*. Stacks examine the item that was most recently added. It is called LIFO (Last In, First Out). On the other hand, a Queue has the operation called *enqueue* to insert an item and *dequeue* to remove an item. It examines the item that was least recently added, which is called FIFO (First In, First Out).

For implementing the Stack we have three methods called: `push()`, `pop()` and `is_empty()`. In push, we insert a new string onto the stack, in pop, we remove and return the string most recently added, and is_empty, we verify if the stack is empty. We still can have a method called `size()` that returns the number of strings on the stack. 

In a first example, we consider that any input is pushed to the stack until a dash is received. When the input corresponds to a dash, we pop the last element of the stack. The image bellow illustrates the process of adding and removing elements of the stack.

<img src="https://cdn.rawgit.com/rogergranada/MOOCs/master/Coursera/Princeton/Algorithms-Part-1/Week%202/images/stack_linkedlist.svg" width="50%" align="center"/>

Thus, we implement the stack using a linked-list representation. In this implementation, we have a node with the *item* containing the string and with a reference to the *next* node. We represent this structure as an inner class as:

```python
class Node:
    def __init__(self):
        item = ''
        next = None
```

In [11]:
class LinkedStackOfStrings(object):
    def __init__(self):
        self.first = None
    
    class Node:
        def __init__(self):
            self.item = ''
            self.next = None

    def is_empty(self):
        return self.first == None

    def push(self, item):
        oldfirst = self.first
        self.first = self.Node()
        self.first.item = item
        self.first.next = oldfirst

    def pop(self):
        item = self.first.item
        self.first = self.first.next
        return item
    
sentence = ['to', 'be', 'or', 'not', '-', '-', 'to', 'be']

stack = LinkedStackOfStrings()
for w in sentence:
    if w == '-' and not stack.is_empty():
        w = stack.pop()
        print('Removed: {}'.format(w))
    else:
        print('Added: {}'.format(w))
        stack.push(w)

print('')
while not stack.is_empty():
    print('Elements: {}'.format(stack.pop()))

Added: to
Added: be
Added: or
Added: not
Removed: not
Removed: or
Added: to
Added: be

Elements: be
Elements: to
Elements: be
Elements: to


Considering the consumption of memory, a stack with N items uses $\sim 40N$ bytes. For each $N$, it includes 16 bytes for object overhead, 8 bytes for inner class extra overhead, 8 bytes for reference to String, and 8 bytes for reference to Node.

### Array Implementation

Instead of using linked-list implementation, we can use an array implementation of a stack. In this sense, we use an array `s[ ]` to store $N$ items on stack. In this implementation, `push()` adds a new item at `s[N]` and `pop()` removes an item from `s[N-1]`. This configuration generates arrays with a certain capacity such as the one below with `capacity=10`:

| 0  | 1  | 2  | 3   | 4  | 5  | 6      | 7      | 8      | 9      |
|----|----|----|-----|----|----|--------|--------|--------|--------|
| to | be | or | not | to | be | *null* | *null* | *null* | *null* |

This kind of implementation has the drawback of occuring stack overflows when $N$ exceeds the capacity in some languages. An implementation using arrays is shown below.

In [24]:
class FixedCapacityStackOfStrings(object):
    def __init__(self, capacity):
        self.s = ['None']*capacity
        self.N = 0
        self.capacity = capacity

    def is_empty(self):
        return self.N == 0

    def push(self, item):
        if self.N+1 == self.capacity:
            raise Exception('ERROR: Cannot add element [Overflow]')
        self.s[self.N] = item
        self.N += 1

    def pop(self):
        if self.is_empty():
            raise Exception('ERROR: Cannot remove element [Underflow]')
        self.N -= 1
        item = self.s[self.N]
        self.s[self.N] = 'None'
        return item

    
sentence = ['to', 'be', 'or', 'not', '-', '-', 'to', 'be']

stack = FixedCapacityStackOfStrings(10)
print('Initial stack: {}'.format(stack.s))
print('')
for w in sentence:
    if w == '-' and not stack.is_empty():
        w = stack.pop()
        print('Removed: {}'.format(w))
    else:
        print('Added: {}'.format(w))
        stack.push(w)
print('')
print('Final stack: {}'.format(stack.s))


Initial stack: ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None']

Added: to
Added: be
Added: or
Added: not
Removed: not
Removed: or
Added: to
Added: be

Final stack: ['to', 'be', 'to', 'be', 'None', 'None', 'None', 'None', 'None', 'None']


**Note**: the way Python deals with arrays, if we create an array with `None` instead of `"None"`, the array contains `size=0` since there is not a valid element in the array. In other languages such as Java, we have to declare the size of the array when creating it, which limits the size of our array. 

## Resizing Array

Resizing an array every time a new element greater than $N$ is added to the array is infeasible for large values of $N$, since it takes time proportional to $1+2+ \ldots +N \sim N^2/2$. Thus, we have to consider a way to ensure that the array resizing occurs infrequently. 

An approach to solve this problem is to create a new array of twice the size of the original array and copy the items every time the original array is full. This is called *repeated doubling*. As consequence, inserting first $N$ items takes time proportional to $N$ and not $N^2$ anymore. 

Unlike `push()`, where every time the array gets a new element that would produce an overflow we create a new array, in `pop()` we do not shrink the array every time the number of elements decreases the half of the size. For example, consider an array of `size=8` where we add a new element. Now we duplicate the original array to `size=16`. If we pop the 9th element, we should not reduce the size of the array to `size=8` again. Otherwise, when we add a new element, we have to duplicate again the array, and copy all elements to the new array again. Thus, we consider reducing the size of the array only when the number of elements is equal to a quarter of the size of the array. Having a quarter of elements, we reduce the size of the array to half of its size. An implementation of the resizing can be seen below.

In [1]:
class ResizingArrayStackOfStrings(object):
    def __init__(self):
        self.s = ['None']
        self.N = 0
        
    def is_empty(self):
        return self.N == 0

    def resize(self, capacity):
        array_copy = ['None']*capacity
        for i in range(self.N):
            array_copy[i] = self.s[i]
        self.s = array_copy[:]
        
    def push(self, item):
        if self.N == len(self.s):
            self.resize(2*len(self.s))
        self.s[self.N] = item
        self.N += 1
        
    def pop(self):
        self.N -= 1
        item = self.s[self.N]
        self.s[self.N] = 'None'

        quarter = len(self.s)/4
        if self.N > 0 and self.N == quarter:
            self.resize(len(self.s)/2)
        return item
    
    
sentence = ['to', 'be', 'or', 'not', 'to', '-', 'be', '-', '-', 'that', '-', '-', '-', 'is']

stack = ResizingArrayStackOfStrings()
print('Initial array: {}'.format(stack.s))
print('')
print('Input\tOutput\tArray content')
print('-----\t------\t-------------')
for w in sentence:
    if w == '-' and not stack.is_empty():
        w = stack.pop()
        print('-\t{}\t{}'.format(w, stack.s))
    else:
        stack.push(w)
        print('{}\t+\t{}'.format(w, stack.s))
print('')
print('Final array: {}'.format(stack.s))



Initial array: ['None']

Input	Output	Array content
-----	------	-------------
to	+	['to']
be	+	['to', 'be']
or	+	['to', 'be', 'or', 'None']
not	+	['to', 'be', 'or', 'not']
to	+	['to', 'be', 'or', 'not', 'to', 'None', 'None', 'None']
-	to	['to', 'be', 'or', 'not', 'None', 'None', 'None', 'None']
be	+	['to', 'be', 'or', 'not', 'be', 'None', 'None', 'None']
-	be	['to', 'be', 'or', 'not', 'None', 'None', 'None', 'None']
-	not	['to', 'be', 'or', 'None', 'None', 'None', 'None', 'None']
that	+	['to', 'be', 'or', 'that', 'None', 'None', 'None', 'None']
-	that	['to', 'be', 'or', 'None', 'None', 'None', 'None', 'None']
-	or	['to', 'be', 'None', 'None']
-	be	['to', 'None']
is	+	['to', 'is']

Final array: ['to', 'is']


The memory usage in this implementation is between $\sim 8N$ (when it is full) and $\sim 32N$ (when it is one-quarter full) bytes to represent a stack with $N$ items. The implementation considers 8 bytes for reference to the array, 24 bytes for array overhead, 8 bytes $\times$ array size for the string array, 4 bytes for int $N$ variable and 4 bytes for padding. This accounts for the memory for the stack and not the memory for string themselves.

### Pros and Cons

- *Linked-list implementation*: Every operation takes constant time in the worst case, and uses extra time and space to deal with the links.
- *Resizing-array implementation*: Every operation takes constant amortized time, and less wasted space.

## Queues

Unkike Stacks, for implementing the Queue we have three methods called: enqueue(), dequeue() and is_empty(). In enqueue, we insert a new string onto the queue, in dequeue, we remove and return the string least recently added, and is_empty, we verify if the queue is empty. We still can have a method called size() that returns the number of strings on the queue. 

In a first example, we consider that any input is enqueued to the queue until a dash is received. When the input corresponds to a dash, we dequeue the element least recently added to the queue. The image bellow illustrates the process of adding and removing elements of the queue.

<img src="https://cdn.rawgit.com/rogergranada/MOOCs/master/Coursera/Princeton/Algorithms-Part-1/Week%202/images/queue_linkedlist.svg" width="50%" align="center"/>

Thus, we implement the queue using a linked-list representation. As the stack implementation, here we have a node with the *item* containing the string and with a reference to the *next* node.

In [58]:
class LinkedQueueOfStrings(object):
    def __init__(self):
        self.first = None
        self.last = None
    
    class Node:
        def __init__(self):
            self.item = ''
            self.next = None

    def is_empty(self):
        return self.first == None

    def enqueue(self, item):
        oldlast = self.last
        self.last = self.Node()
        self.last.item = item
        self.last.next = None
        if self.is_empty():
            self.first = self.last
        else:
            oldlast.next = self.last

    def dequeue(self):
        item = self.first.item
        self.first = self.first.next
        if self.is_empty():
            self.last = None
        return item
    
sentence = ['to', 'be', 'or', 'not', '-', '-', 'to', 'be']

queue = LinkedQueueOfStrings()
for w in sentence:
    if w == '-' and not queue.is_empty():
        w = queue.dequeue()
        print('Removed: {}'.format(w))
    else:
        queue.enqueue(w)
        print('Added: {}'.format(w))

print('')
while not queue.is_empty():
    print('Elements: {}'.format(queue.dequeue()))

Added: to
Added: be
Added: or
Added: not
Removed: to
Removed: be
Added: to
Added: be

Elements: or
Elements: not
Elements: to
Elements: be


### Array Implementation

As occurred in stack, here we can use an array implementation of a queue. We use an array `s[ ]` to store $N$ items on the queue. In this implementation, `enqueue()` adds a new item at `s[tail++]` increasing the `tail` of our queue and `dequeue()` removes an item from `s[head++]`. This configuration generates arrays with a certain capacity such as the one below with `capacity=10`. Consider in the example below that first, we add the elements *to*, *be*, *or*, *not*. Then, we remove (dequeue) 2 elements (*to*) and (*be*), and finally add two more elements *to* and *be*. As we add and remove elements of the queue, we can see the `head` and the `tail` increasing. When we have such structure, we have to move all elements to the `index=0` when our `tail` overflows and our `head` is greater than zero.

| 0      | 1      | 2      | 3      | 4      | 5      | 6      | 7      | 8      | 9      | $\ $ | Head | Tail |
|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|------|------|------|
| to     | *null* | *null* | *null* | *null* | *null* | *null* | *null* | *null* | *null* | $\ $ |  0   |  1   |
| to     | be     | *null* | *null* | *null* | *null* | *null* | *null* | *null* | *null* | $\ $ |  0   |  2   |
| to     | be     | or     | *null* | *null* | *null* | *null* | *null* | *null* | *null* | $\ $ |  0   |  3   |
| to     | be     | or     | not    | *null* | *null* | *null* | *null* | *null* | *null* | $\ $ |  0   |  4   |
| *null* | be     | or     | not    | *null* | *null* | *null* | *null* | *null* | *null* | $\ $ |  1   |  4   |
| *null* | *null* | or     | not    | *null* | *null* | *null* | *null* | *null* | *null* | $\ $ |  2   |  4   |
| *null* | *null* | or     | not    | to     | *null* | *null* | *null* | *null* | *null* | $\ $ |  2   |  5   |
| *null* | *null* | or     | not    | to     | be     | *null* | *null* | *null* | *null* | $\ $ |  2   |  6   |

This kind of implementation has the drawback of occuring queue overflows when $N$ exceeds the capacity in some languages. An implementation using arrays is shown below.

In [57]:
class FixedCapacityQueueOfStrings(object):
    def __init__(self, capacity):
        self.s = ['None']*capacity
        self.capacity = capacity
        self.head = 0
        self.tail = 0

    def is_empty(self):
        return self.head == self.tail
    
    def move_elements(self):
        for i, ie in enumerate(range(self.head, self.tail)):
            self.s[i] = self.s[ie]
            self.s[ie] = 'None'
        self.head = 0
        self.tail = i+1

    def enqueue(self, item):
        if self.tail == self.capacity: 
            if self.head == 0:
                raise Exception('ERROR: Cannot add element [Overflow]')
            else:
                self.move_elements()
        self.s[self.tail] = item
        self.tail += 1

    def dequeue(self):
        if self.is_empty():
            raise Exception('ERROR: Cannot remove element [Underflow]')
        item = self.s[self.head]
        self.s[self.head] = 'None'
        self.head += 1
        return item

    
sentence = ['to', 'be', 'or', 'not', '-', '-', 'to', 'be']
queue = FixedCapacityQueueOfStrings(5)
print('Initial array: {}'.format(queue.s))
print('')
print('Input\tOutput\tArray content')
print('-----\t------\t-------------')
for w in sentence:
    if w == '-' and not queue.is_empty():
        w = queue.dequeue()
        print('-\t{}\t{}'.format(w, queue.s))
    else:
        queue.enqueue(w)
        print('{}\t+\t{}'.format(w, queue.s))
print('')
print('Final array: {}'.format(queue.s))

Initial array: ['None', 'None', 'None', 'None', 'None']

Input	Output	Array content
-----	------	-------------
to	+	['to', 'None', 'None', 'None', 'None']
be	+	['to', 'be', 'None', 'None', 'None']
or	+	['to', 'be', 'or', 'None', 'None']
not	+	['to', 'be', 'or', 'not', 'None']
-	to	['None', 'be', 'or', 'not', 'None']
-	be	['None', 'None', 'or', 'not', 'None']
to	+	['None', 'None', 'or', 'not', 'to']
be	+	['or', 'not', 'to', 'be', 'None']

Final array: ['or', 'not', 'to', 'be', 'None']


# Generics in Java

When programming in Java, we can create stacks or queues for different types of objects. Thus, we can create a stack named `StackOfStrings`, another named `StackOfInts`, another `StackOfURLs` and so on. However we can see that the unique diference between these stacks is the type of object they are using. Instead of creating a stack for each type of object, we can create a generic stack and only casting or declare its type for different types of objects. 

For example, we can use casting to the object that we are retrieving from a list. The problem is that in case of adding a different type in the list, we can discover the error only in runtime. The example below shows the casting for object.

```java
StackOfObjects s = new StackOfObjects();
Apple a = new Apple();
Orange b = new Orange();
s.push(a);
s.push(b);
a = (Apple) (s.pop()); //Run-time error
```

When declaring different types of objects explicitly, as in the example below, we can identify error in compile-time instead of run-time. To do so in Java, we declare the type parameters as follows:

```java
Stack<Apple> s = new Stack<Apple>();
Apple a = new Apple();
Orange b = new Orange();
s.push(a);
s.push(b); //Compile-time error
a = s.pop();
```

Thus, our linked-list implementation would become in Java:

```java
public class Stack<Item> {       //<Item> is a generic type name
    private Node first = null;

    private class Node {
        Item item;               //generic type name
        Node next;
    }

    public boolean isEmpty(){
        return first == null;
    }

    public void push(Item item){ //genreric type name
        Node oldfirst = first;
        first = new Node();
        first.item = item;
        first.next = oldfirst;
    }

    public Item pop(){          //generic type name
        Item item = first.item; //generic type name
        first = first.next;
        return item;
    }
}
```

It is important to note that when implementing lists of objects, we cannot use the declaration as:

```java
private Item[] s;
...
s = new Item[capacity];
```

Since it would lead to a compile-time error. Instead, we have to use casting to declare the type of object as:

```java
private Item[] s;
...
s = (Item[]) new Object[capacity];
```

Unlike non-primitive types, primitive types have a wrapper that automatically assign the type for the variable without needing to do casting. For example, consider the code below:

```java
Stack<Integer> s = new Stack<Integer>();
s.push(17);       // equals to: s.push(Integer.valueOf(17));
int a = s.pop();  // equals to: int a = s.pop().intValue()
```

It is important to say here that we use Java to illustrate generic types since Python has *Duck typing*. *i.e.*, "If it walks like a duck and it quacks like a duck, then it must be a duck". Thus, it can "discover" the type of the object automatically without needing the casting or declaring the type explicitly.

## Iterators

Iterators allow to iterate over the items by a client without revealing the internal representation of the stack or queue. In Java, an object is iterable when it has a method that returns an iterator. In order to implement it, we must have a method called `has_next()` and another called `next()`. In the former, we verify whether exists a next item in the stack/queue. In the later, we return the next item of the stack/queue. As an example, in Java we have the following code that implements the iterator:

```java
import java.util.Iterator;

public class Stack<Item> iterator() {
    return new ListIterator();


    private class ListIterator implements Iterator<Item> {
        private Node current = first;
     
        private boolean hasNext() {
            return current != null;
        }

        public Item next() {
            Item item = current.item;
            current = current.next;
            return item;
        }
    }
}
```

## Bag API

Bag API works by adding items to a collection and iterating over them without considering the order of the elements. The image below illustrates how Bag API works:

<img src="https://cdn.rawgit.com/rogergranada/MOOCs/master/Coursera/Princeton/Algorithms-Part-1/Week%202/images/bag_api.svg" width="60%" align="center"/>

In Java, its implementation is as follows:

```java
public class Bag<Item> implements Iterable<Item>{
    Bag()                     //create an empty bag
    void add(Item x)          //inserting a new item onto the bag
    int size()                //number of items in bag
    Iterable<Item> iterator() //iterator for all items in bag
```

## Applications

A Stack can be used for arithmetic expression evaluation, where the goal is to evaluate infix expressions. For example, the Two-Stack Algorithm by E. W. Dijkstra. In this algorithm, we evaluate infix expressions by manipulate elements surrounded by parentheses using two stacks: the *value stack* and the *operator stack*. The *value stack* maintains numbers of the expressions and the *operator stack* maintains the operators of the operations. In this algorithm, when we find a left parenthesis `(` no action is taken, when we find a number, it is pushed onto the *value stack*, when we find an operator, it is pushed onto the *operator stack*, and when we find a right parenthesis `)`, we pop two values and one operator, perform their operation and push the resulting value back onto the *value stack*. An example of the operation `( 1 + ( ( 2 + 3 ) * ( 4 * 5 ) ) ))` is illustrated below: 

<img src="https://cdn.rawgit.com/rogergranada/MOOCs/master/Coursera/Princeton/Algorithms-Part-1/Week%202/images/two_stack_algorithm.svg" width="80%" align="center"/>

This algorithm works by solving inner expressions to outer expressions. Thus, when the algorithm encounters an operator surrounded by two values within a parentheses, it leaves the result on the value stack. This process is repeated up to the last parenthesis, solving the equation.

A Python implementation of this algorithm can be seen below.

In [5]:
def evaluate(expression):
    value_stack = ResizingArrayStackOfStrings()
    operator_stack = ResizingArrayStackOfStrings()
    
    for v in expression:
        if v == '(' or v == ' ':
            continue
        elif v in ['+', '-', '*', '/']:
            operator_stack.push(v)
        elif v == ')':
            op = operator_stack.pop()
            if op == '+': 
                value_stack.push(value_stack.pop() + value_stack.pop())
            elif op == '-': 
                value_stack.push(value_stack.pop() - value_stack.pop())
            elif op == '*': 
                value_stack.push(value_stack.pop() * value_stack.pop())
            elif op == '/': 
                value_stack.push(value_stack.pop() / value_stack.pop())
        else:
            value_stack.push(int(v))
    return value_stack.pop()

exp = '( 1 + ( ( 2 + 3 ) * ( 4 * 5 ) ) ))'
print(evaluate(exp))

101


# Questions

1. Which of the following inputs to our stack test client does not produce the output `5 4 3 2 1`?<br>
&#9744; 1 2 3 4 5 - - - - -<br>
&#9744; 1 2 5 - 3 4 - - - -<br>
&#9745; 5 - 1 2 3 - 4 - - -<br>
&#9744; 5 - 4 - 3 - 2 - 1 -

2. Given a reference `first` to the first node of a null-terminated linked list with at least two nodes, what does the code fragment below do?


```python
Node x = first;
while (x.next.next != null) {
    x = x.next;
}
x.next = null;
```

&#9744; deletes the first node in the list<br>
&#9744; deletes the second node in the list<br>
&#9744; deletes the next-to-last node in the list<br>
&#9745; deletes the last node in the list

3. Suppose that, starting from an empty data structure, we perform $n$ push operations in our resizing-array implementation of a stack. How many times is the `resize()` method called?<br>

&#9744; constant<br>
&#9745; logarithmic<br>
&#9744; linear<br>
&#9744; quadratic



4. Suppose that you implement a queue using a null-terminated singly-linked list, maintaining a reference to the item least recently added (the front of the list) but not maintaining a reference to the item most recently added (the end of the list). What are the worst-case running times for enqueue and dequeue?<br>

&#9744; constant time for both enqueue and dequeue<br>
&#9744; constant time for enqueue and linear time for dequeue<br>
&#9745; linear time for enqueue and constant time for dequeue<br>
&#9744; linear time for both enqueue and dequeue

5. Which of the following statements is a type safe way to declare and initialize a `Stack` of integers in Java?<br>

&#9744; `Stack<int> stack = new Stack<int>();`<br>
&#9744; `Stack<Integer> stack = new Stack();`<br>
&#9744; `Stack stack = new Stack<Integer>();`<br>
&#9745; `Stack<Integer> stack = new Stack<Integer>();`

6. Suppose that we copy the iterator code from our linked list and resizing array implementations of a stack to the corresponding implementations of a queue. Which queue iterator(s) will correctly return the items in FIFO order?<br>

&#9744; neither<br>
&#9745; linked-list iterator only<br>
&#9744; array iterator only<br>
&#9744; both

**Answer**: The linked-list iterator will work without modification because the items in the linked list are ordered in FIFO order (which is the main reason we dequeue from the front and enqueue to the back instead of vice versa). The array iterator will fail for two reasons. First, the the items should be iterated over in the opposite order. Second, the items will not typically be stored in the array as entries $0$ to $n−1$.

7. What does the following code fragment print?<br>

```java
int n = 50;
Stack<Integer> stack = new Stack<Integer>();

while (n > 0) {
    stack.push(n % 2);
    n = n / 2;
}
for (int digit : stack) {
    StdOut.print(digit);
}
StdOut.println();
```

&#9744; 010011<br>
&#9744; 010111<br>
&#9744; 111010<br>
&#9745; 110010 

**Answer**: The first step is to stack only resulting values of the division by two (digit 0 or 1). Thus, we have the while over the values `50 25 12 6 3 1` with the resulting division by 2 `0 1 0 0 1 1`. As the elements are stored onto the stack, the order in which they are retrieved is the oposite as they are inserted, and thus, the correct answer is `1 1 0 0 1 1`. 

# Interview Questions: Stacks and Queues

1. **Queue with two stacks**. Implement a queue with two stacks so that each queue operations takes a constant amortized number of stack operations.

**Answer**: We can use two stacks to create a queue. In our implementation, one stack will serve to `enqueue` and the other to `dequeue` elements. We only change elements from one stack to the other when the stack of `dequeue` contains zero elements and we still have elements in the `enqueue` stack. The figure below illustrates the process of enqueue and dequeue using the two stacks.

<img src="https://cdn.rawgit.com/rogergranada/MOOCs/master/Coursera/Princeton/Algorithms-Part-1/Week%202/images/2stack_queue.svg" width="100%" align="center"/>

Below the code to run a queue using two stacks.

In [6]:
class Queue2Stacks(object):
    def __init__(self):
        self.st_enqueue = ResizingArrayStackOfStrings()
        self.st_dequeue = ResizingArrayStackOfStrings()
        self.size = 0
        
    def enqueue(self, item):
        self.st_enqueue.push(item)
        self.size += 1
        
    def dequeue(self):
        if self.st_dequeue.is_empty():
            if self.st_enqueue.is_empty():
                raise Exception('ERROR: Cannot retrieve element [Underflow]')
            else:
                while not self.st_enqueue.is_empty():
                    self.st_dequeue.push(self.st_enqueue.pop())
        self.size -= 1
        return self.st_dequeue.pop()
        
    def is_empty(self):
        if self.st_enqueue.is_empty() and self.st_dequeue.is_empty():
            return True
        return False
    
    def size(self):
        return self.size

values = [0, 1, 2, 3]
q2s = Queue2Stacks()
q2s.enqueue(0)
q2s.enqueue(1)
q2s.enqueue(2)
print('Dequeue 1st item: {}'.format(q2s.dequeue()))
print('Dequeue 2nd item: {}'.format(q2s.dequeue()))
q2s.enqueue(3)
print('Dequeue 3rd item: {}'.format(q2s.dequeue()))
print('Dequeue 4th item: {}'.format(q2s.dequeue()))

Dequeue 1st item: 0
Dequeue 2nd item: 1
Dequeue 3rd item: 2
Dequeue 4th item: 3


2. **Stack with max**. Create a data structure that efficiently supports the stack operations (push and pop) and also a return-the-maximum operation. Assume the elements are real numbers so that you can compare them.

**Answer**: For this structure, called StackWithMax, we have a normal stack that manipulates all items (push and pop operations), and another stack to control the maximum values. For each new item added in the regular stack, this stack (hereafter called max stack) will add a new element too. This element is the maximum value between the value of the element in the top and the new item to be added. When we pop the regular stack, the max stack also has its item removed from the top. The image below illustrates the push and pop operations for the new data structure considering both stacks.  

<img src="https://cdn.rawgit.com/rogergranada/MOOCs/master/Coursera/Princeton/Algorithms-Part-1/Week%202/images/max_stack.svg" width="100%" align="center"/>

Below the code that implements the StackWithMax structure.

In [ ]:
class StackWithMax(object):
    def __init__(self):
        self.st_regular = ResizingArrayStackOfStrings()
        self.st_max = ResizingArrayStackOfStrings()
        self.size = 0
        
    def enqueue(self, item):
        self.st_enqueue.push(item)
        self.size += 1
        
    def dequeue(self):
        if self.st_dequeue.is_empty():
            if self.st_enqueue.is_empty():
                raise Exception('ERROR: Cannot retrieve element [Underflow]')
            else:
                while not self.st_enqueue.is_empty():
                    self.st_dequeue.push(self.st_enqueue.pop())
        self.size -= 1
        return self.st_dequeue.pop()
        
    def is_empty(self):
        if self.st_enqueue.is_empty() and self.st_dequeue.is_empty():
            return True
        return False
    
    def size(self):
        return self.size

3. **Java generics**. Explain why Java prohibits generic array creation.

**Answer**: Java prohibits generic array creation because it implements Generics on the compiler level and the information of type parameters is discarded by the compiler after compilation. As types are discarded, it is not available at runtime, which means that when creating a generic array the runtime cannot know the type of the array, generating an error. 